### CSC2611 Exercise: Meaning construction from text

Self-assessment/precursor exercise to assignment

In [1]:
# importing all libraries

import pandas as pd
import pickle
import numpy as np
from numpy import errstate

#Q1
import nltk

from nltk.corpus import brown
from collections import Counter
import string

In [2]:
# for formatting
%load_ext blackcellmagic

#### Q2: Extract the 5000 most common English words (denoted by W) based on unigram frequencies in the Brown corpus.  Report the 5 most and least common words you have found in the 5000 words.  Update W by adding n words where n is the set of words in Table 1of RG65 that were not included in the top 5000 words from the Brown corpus.  Denote the total number of words in W as |W|.

Note: Including stopwords in the English corpus, but removing punctuations such as "`" present in the corpus + lowercasing all values

In [3]:
# getting all Brown words; brown.words() returns an iterator
words_iter = brown.words()
brown_words = [word.lower() for word in words_iter if (word not in string.punctuation)]

# getting unigram counts
uni_counts = Counter(brown_words)

# getting most common 5000
top_5000 = uni_counts.most_common(5000)

# top 5000 words
W = [word for (word, count) in top_5000]

In [4]:
# Top 500 words
print("5 most common in top 5000", top_5000[:5])
print("5 least common in top 5000", top_5000[-5:])

5 most common in top 5000 [('the', 69971), ('of', 36412), ('and', 28853), ('to', 26158), ('a', 23195)]
5 least common in top 5000 [('input', 20), ('suitcase', 20), ('skyros', 20), ('freddy', 20), ('airport', 19)]


###### Adding words from RG65

In [5]:
rg_words_first = [
    "cord",
    "hill",
    "rooster",
    "car",
    "noon",
    "cemetery",
    "fruit",
    "glass",
    "autograph",
    "magician",
    "automobile",
    "crane",
    "mound",
    "brother",
    "grin",
    "sage",
    "asylum",
    "oracle",
    "asylum",
    "bird",
    "graveyard",
    "bird",
    "glass",
    "food",
    "boy",
    "brother",
    "cushion",
    "asylum",
    "monk",
    "furnace",
    "asylum",
    "magician",
    "coast",
    "hill",
    "grin",
    "cord",
    "shore",
    "glass",
    "monk",
    "grin",
    "boy",
    "serf",
    "automobile",
    "journey",
    "mound",
    "autograph",
    "lad",
    "coast",
    "forest",
    "forest",
    "food",
    "implement",
    "cemetery",
    "cock",
    "shore",
    "boy",
    "bird",
    "cushion",
    "coast",
    "cemetery",
    "furnace",
    "automobile",
    "crane",
    "midday",
    "gem",
]
rg_words_second = [
    "smile",
    "woodland",
    "voyage",
    "journey",
    "string",
    "mound",
    "furnace",
    "jewel",
    "shore",
    "oracle",
    "wizard",
    "implement",
    "stove",
    "lad",
    "implement",
    "wizard",
    "fruit",
    "sage",
    "monk",
    "crane",
    "madhouse",
    "cock",
    "magician",
    "fruit",
    "rooster",
    "monk",
    "jewel",
    "madhouse",
    "slave",
    "stove",
    "cemetery",
    "wizard",
    "forest",
    "mound",
    "lad",
    "string",
    "woodland",
    "tumbler",
    "oracle",
    "smile",
    "sage",
    "slave",
    "cushion",
    "voyage",
    "shore",
    "signature",
    "wizard",
    "shore",
    "graveyard",
    "woodland",
    "rooster",
    "tool",
    "woodland",
    "rooster",
    "voyage",
    "lad",
    "woodland",
    "pillow",
    "hill",
    "graveyard",
    "implement",
    "car",
    "rooster",
    "noon",
    "jewel",
]
sim_scores = [
    0.02,
    1.48,
    0.04,
    1.55,
    0.04,
    1.69,
    0.05,
    1.78,
    0.06,
    1.82,
    0.11,
    2.37,
    0.14,
    2.41,
    0.18,
    2.46,
    0.19,
    2.61,
    0.39,
    2.63,
    0.42,
    2.63,
    0.44,
    2.69,
    0.44,
    2.74,
    0.45,
    3.04,
    0.57,
    3.11,
    0.79,
    3.21,
    0.85,
    3.29,
    0.88,
    3.41,
    0.9,
    3.45,
    0.91,
    3.46,
    0.96,
    3.46,
    0.97,
    3.58,
    0.97,
    3.59,
    0.99,
    3.6,
    1.0,
    3.65,
    1.09,
    3.66,
    1.18,
    3.68,
    1.22,
    3.82,
    1.24,
    3.84,
    1.26,
    3.88,
    1.37,
    3.92,
    1.41,
    3.94,
    3.94,
]


rg_all_words = []
rg_all_words.extend(rg_words_first)
rg_all_words.extend(rg_words_second)
rg_all_words = list(set(rg_words_first))

# assert all lists have same number of values, i.e., 65
assert len(rg_words_first)==len(rg_words_second)==len(sim_scores)==65


# getting number of words not in n
n = len(set(rg_all_words)) - len(set(rg_all_words).intersection(W))

# getting words not in W already, but present in RG65 words
new_rg_words = list(set(rg_all_words) - set(rg_all_words).intersection(W))

W.extend(new_rg_words)

mod_W = len(W)
print("|W|=", mod_W)

|W|= 5021


#### Q4: Construct a word-context vector model (denoted by M1) by collecting bigram counts for words in W

In [6]:
# to get bigrams
from nltk.util import ngrams

# to build a sparse matrix
from scipy import sparse

In [7]:
bigram_list = list(ngrams(words_iter, 2))

In [8]:
# getting a mapping from words to indices, or vocabulary IDs
vocab_to_id = {}
for i, word in enumerate(W):
    vocab_to_id[word] = i

# Saving the dictionary for future exercise
output = open("../lab/data/lsa_vocab_to_id.pkl", "wb")
pickle.dump(vocab_to_id, output)

In [9]:
# constructing M1
# |W|x|W| matrix
M1 = np.zeros((mod_W, mod_W))

for (first_word, second_word) in bigram_list:
    try:
        M1[vocab_to_id[first_word]][vocab_to_id[second_word]] += 1
    except KeyError as e:
        pass
M1 = sparse.csr_matrix(M1)

In [10]:
# compute ppmi
import math
def compute_ppmi(M1):
    normalization_constant = np.sum(M1)
    pmi_before_log = M1.copy()
    
    p_i = np.sum(M1,axis=0)/normalization_constant
    p_j = np.sum(M1,axis=1)/normalization_constant
    
    # p(i)*p(j)
    p_i_into_p_j = np.outer(p_i,p_j)
    
    div_factor = p_i_into_p_j
    pmi_before_log = M1/(p_i_into_p_j*normalization_constant)

    # setting NaN to 0
    pmi_before_log[np.isnan(pmi_before_log)] = 0
    
    # only considering positive infinity since all
    # values in matrix are positive frequencies
    pmi_before_log[~np.isfinite(pmi_before_log)] = 1.7976931348623157e+308

    with errstate(divide='ignore'):
        pmi = np.log(pmi_before_log)

    ppmi = pmi.copy()

    # returning POSITIVE PMI
    ppmi[ppmi < 0] = 0.0

    return ppmi

In [11]:
M1_plus = compute_ppmi(M1)

In [12]:
M1_plus

matrix([[0.        , 0.        , 0.        , ..., 2.7147188 , 2.20389317,
         0.        ],
        [1.6717954 , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.90421793, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 1.82589416, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [1.2899834 , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

### Q5: Construct  a  latent  semantic  model using PCA with varying dimensions

In [13]:
from sklearn.decomposition import PCA

In [14]:
lsa_10 = PCA(n_components=10)
lsa_100 = PCA(n_components=100)
lsa_300 = PCA(n_components=300)

M2_10 = lsa_10.fit_transform(M1_plus)
M2_100 = lsa_100.fit_transform(M1_plus)
M2_300 = lsa_300.fit_transform(M1_plus)

# Saving the dictionary for future exercise
output = open("../lab/data/lsa_300.pkl", "wb")
pickle.dump(M2_300, output)

### Q6: Find all pairs of words in Table 1 of RG65 that are also available in W.  Denote these pairs as P.  Record the human-judged similarities of these word pairs from the table and denote similarity values as S

In [15]:
# finding all word-pairs such that both words in pair are in W

P = []
S = []
count = 0
for first_word, second_word in zip(rg_words_first, rg_words_second):
    if (first_word in W) and (second_word in W):
        P.append((first_word, second_word))
        S.append(sim_scores[count])
    count+=1

### Q7: Perform the following calculations on each of these models M1,M1+, M210, M2100,M2300, separately:  Calculate cosine similarity between each pair of words in P, based on the constructed word vectors.  

In [16]:
# to comput similarity
from sklearn.metrics.pairwise import cosine_similarity



In [17]:
sm1 = []
sm1_plus = []
sm2_10 = []
sm2_100 = []
sm2_300 = []
for i, (first_word, second_word) in enumerate(P):
    first_id = vocab_to_id[first_word]
    second_id = vocab_to_id[second_word]

    sm1.append(cosine_similarity(M1[first_id], M1[second_id])[0][0])
    sm1_plus.append(cosine_similarity(M1_plus[first_id], M1_plus[second_id])[0][0])
    sm2_10.append(cosine_similarity([M2_10[first_id]], [M2_10[second_id]])[0][0])
    sm2_100.append(cosine_similarity([M2_100[first_id]], [M2_100[second_id]])[0][0])
    sm2_300.append(cosine_similarity([M2_300[first_id]], [M2_300[second_id]])[0][0])

In [18]:
sim_names = ["SM1", "SM1_plus", "SM2_10", "SM2_100", "SM2_300"]
sim_arrs = [sm1, sm1_plus, sm2_10, sm2_100, sm2_300]
for sim_name, sim_arr in zip(sim_names, sim_arrs):
    print(sim_name, sim_arr)
    print("*" * 20)

SM1 [0.0, 0.38919299029595256, 0.0, 0.31430927854685603, 0.0, 0.0, 0.5, 0.1770844008302866, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12700012700019048, 0.39466488146729556, 0.0, 0.049386479832479485, 0.048795003647426664, 0.0, 0.3571973786082756, 0.0, 0.0, 0.0, 0.4226664576955308, 0.0, 0.0, 0.0, 0.20025046972870353, 0.27773983045116846, 0.0, 0.0, 0.0, 0.0, 0.3684529491774706, 0.10088665446106368, 0.2886751345948129, 0.0, 0.4474063620329013, 0.0, 0.26726124191242434]
********************
SM1_plus [0.0, 0.05846616937858707, 0.0, 0.03052650650178733, 0.0, 0.0, 0.37271372107998707, 0.06799305472679423, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004768327526323783, 0.06789362325137147, 0.0, 0.0, 0.035824001987207525, 0.0, 0.09051163400453825, 0.0, 0.0, 0.0, 0.12309439071203251, 0.0, 0.0, 0.0, 0.0026153643644516, 0.007467790041795627, 0.0, 0.0, 0.0, 0.0, 0.07486335267152128, 0.009753530236653884, 0.05612817129282398, 0.0, 0.03741200839645414, 0.0, 0.06451776254201196]
********************
SM2_10 [-0.11533849471

### Q8: Report Pearson correlation between S and each of the model-predicted similarities

In [19]:
from scipy.stats import pearsonr

In [20]:
sim_names = ["SM1", "SM1_plus", "SM2_10", "SM2_100", "SM2_300"]
sim_arrs = [sm1, sm1_plus, sm2_10, sm2_100, sm2_300]

for sim_name, sim_arr in zip(sim_names, sim_arrs):
    stat, p = pearsonr(S, sim_arr)
    print(
        "Pearson correlation between S and {} is {}, with p-value {}".format(
            sim_name, stat, p
        )
    )

Pearson correlation between S and SM1 is 0.383297471392835, with p-value 0.013372475262142703
Pearson correlation between S and SM1_plus is 0.23415928136700334, with p-value 0.1406007563000264
Pearson correlation between S and SM2_10 is 0.10749842758503675, with p-value 0.5035071277381504
Pearson correlation between S and SM2_100 is 0.36970425087090264, with p-value 0.017358390444229956
Pearson correlation between S and SM2_300 is 0.35839609577091425, with p-value 0.0213966431652426
